## Calibration of model parameters (BLACK and SCHOLES, MERTON)
The **Implied Volatility** is that value $\\sigma$ that must be inserted into the Black-Scholes (BS) formula in order to retrieve the option price quoted in the market:
$$ BS(S, K, T, r, \sigma) = P  $$
where $S$ is the underlying spot price, $K$ is the strike, $T$ time to maturity, $r$ risk-free interest rate and $P$ the option price quoted in the market. All these quantities are **observable**.


In [29]:
from functions.MERTONpricer import Merton_pricer
from functions.BSpricer import BS_pricer

import numpy as np
import pandas as pd
import scipy as scp
import scipy.stats as ss
import scipy.optimize as scpo


Let's retrieve the historic prices for european call and put options starting from 2016-01-20 and expiring 1 year later.

In [30]:
df_call = pd.read_csv('data/options_spx_call_2016.csv')
df_put = pd.read_csv('data/options_spx_put_2016.csv')

df_call['C_Midpoint'] = abs(df_call['C_BID'] + df_call['C_ASK'] )/ 2
df_put['P_Midpoint'] = abs(df_put['P_BID'] + df_put['P_ASK'] )/ 2

r = 0.1
sigma = 0.5
S0 = df_call.iloc[0]['UNDERLYING_LAST']
T = 1
call_strikes = df_call['STRIKE']    # array of K
put_strikes = df_put['STRIKE']
exercise = 'european'

call_prices = df_call['C_Midpoint']
put_prices = df_put['P_Midpoint']

#print(df_call)

Now let's initialize an object of class BS_pricer which is able to find the theoretical price of the options, given the parameters.

In [31]:
BS_C = BS_pricer(S0=S0, r=r, sigma=sigma, ttm=T, exercise=exercise, K=None, type_o='call')
MertonC = Merton_pricer(S0=S0, K=None, ttm=T, r=r, sigma=sigma, lambd=0.8, meanJ=0.6, stdJ=0.2, exercise=exercise, type_o='call')

BS_call_prices = np.zeros_like(call_strikes, dtype=float)
for i, K in enumerate(call_strikes):
    BS_call_prices[i] = BS_C.closed_formula(K)

MERT_call_prices = np.zeros_like(call_strikes, dtype=float)
for i, K in enumerate(call_strikes):
    MERT_call_prices[i] = MertonC.closed_formula(K)

print(f'Theoretical call options prices [Black and Scholes model]: {BS_call_prices}')
print(f'Theoretical call options prices [Merton Jump Diffusion model]: {MERT_call_prices}')


Theoretical call options prices [Black and Scholes model]: [519.26777644 506.92447997 494.84144177 483.01557869 471.44368111
 460.12242595 449.04838896 438.21805656 427.62783697 417.27407075
 407.15304077 397.26098163 387.5940885  378.14852541]
Theoretical call options prices [Merton Jump Diffusion model]: [621.72072212 610.29182993 599.05279923 588.00157246 577.13606165
 566.45415143 555.953702   545.63255197 535.48852105 525.51941267
 515.72301647 506.09711066 496.63946432 487.3478396 ]


PUT PRICES:

In [32]:
BS_P = BS_pricer(S0=S0, r=r, sigma=sigma, ttm=T, exercise=exercise, K=None, type_o='put')
MertonP = Merton_pricer(S0=S0, K=None, ttm=T, r=r, sigma=sigma, lambd=0.8, meanJ=0.6, stdJ=0.2, exercise=exercise, type_o='put')

BS_put_prices = np.zeros_like(put_strikes, dtype=float)
for i, K in enumerate(put_strikes):
    BS_put_prices[i] = BS_P.closed_formula(K)

MERT_put_prices = np.zeros_like(put_strikes, dtype=float)
for i, K in enumerate(put_strikes):
    MERT_put_prices[i] = MertonP.closed_formula(K)

print(f'Computed put options prices [Black and Scholes model]: {BS_put_prices}')
print(f'Computed put options prices [Merton Jump Diffusion model]: {MERT_put_prices}')


Computed put options prices [Black and Scholes model]: [198.0113871  208.28902609 218.82692333 229.6219957  240.67103358
 251.97071386 263.51761232 275.30821538 287.33893124 299.60610047
 312.10600594 324.83488225 337.78892457 350.96429694]
Computed put options prices [Merton Jump Diffusion model]: [300.46433278 311.65637604 323.03828079 334.60798948 346.36341412
 358.30243934 370.42292536 382.72271078 395.19961532 407.85144239
 420.67598164 433.67101128 446.83430039 460.16361112]


In [33]:
def implied_volatility(price, S0, K, T, r, type_o, method, disp=True ):
    """ Returns Implied volatility
        methods:  fsolve (default) or brent
    """

    def obj_fun(vol):
        return price - BS_C.BlackScholes(type_o=type_o, S0=S0, K=K, ttm=T, r=r, sigma=vol)

    if method == 'brent':
        x, r = scpo.brentq( obj_fun, a=1e-15, b=500, full_output=True)
        if r.converged:
            return x

    if method =='fsolve':
        X0 = [0.03, 0.08, 0.1, 0.15, 0.2, 0.3, 0.5, 1, 2]   # set of initial guess points for imp.vol.
        for x0 in X0:
            x, _, solved, _ = scpo.fsolve(obj_fun, x0, full_output=True, xtol=1e-8)
            if solved == 1:
                return x[0]

    if method == 'newton':
        x0 = 0.3

        def obj_fun_squared(vol):
            return obj_fun(vol)**2

        # print('[σ] = ', x0, "Objective Function value: ", obj_fun_squared(x0))
        result = scpo.fmin(obj_fun_squared, x0, ftol=1e-10, full_output=0, disp=0)
        return result[0]


    if disp:
        return -1

Let's now compute the implied volatilities from the true market prices *call_prices*.

In [34]:
IVs_BS_call = []; IVs_MERT_call = [];
for i in range(len(call_prices)):
    IVs_BS_call.append(implied_volatility(call_prices[i], S0=S0, K=call_strikes[i], T=T, r=r, type_o='call', method='newton') )
    IVs_MERT_call.append(implied_volatility(MERT_call_prices[i], S0=S0, K = call_strikes[i], T=T, r=r, type_o='call', method='fsolve'))

print(f'Implied volatilities [Black and Scholes]: {IVs_BS_call}\n')
print(f'Implied volatilities [Merton Jump Diffusion]: {IVs_MERT_call}')

Implied volatilities [Black and Scholes]: [0.014999999999999208, 0.014999999999999208, 0.014999999999999208, 0.014999999999999208, 0.014999999999999208, 0.014765624999999208, 0.007499999999999188, 0.007499999999999188, 0.007499999999999188, 0.007499999999999188, 0.0037499999999991776, 0.04645385742187429, 0.057302570343016904, 0.0637830448150628]

Implied volatilities [Merton Jump Diffusion]: [0.6672784188699085, 0.6662112902897452, 0.6651620568297247, 0.6641302856710486, 0.6631155585401314, 0.6621174709777291, 0.661135631661977, 0.6601696617803262, 0.6592191944459197, 0.658283874154345, 0.6573633562771634, 0.6564573065889482, 0.6555654008249056, 0.6546873242664388]


In [35]:
IVs_BS_put = []; IVs_MERT_put = [];
for i in range(len(put_prices)):
    IVs_BS_put.append(implied_volatility(put_prices[i], S0=S0, K=put_strikes[i], T=T, r=r, type_o='put', method='fsolve') )
    IVs_MERT_put.append(implied_volatility(MERT_put_prices[i], S0=S0, K = call_strikes[i], T=T, r=r, type_o='put', method='fsolve'))

print(f'Implied volatilities [Black and Scholes]: {IVs_BS_put}')
print(f'Implied volatilities [Merton Jump Diffusion]: {IVs_MERT_put}')

Implied volatilities [Black and Scholes]: [0.35229363616294335, 0.35188275048143236, 0.3506820060204052, 0.35017099337544816, 0.35014355407871467, 0.3502933001043605, 0.35092279692388245, 0.3517300535108708, 0.3530935523924029, 0.3547251036727889, 0.35691040121733497, 0.3595598971892729, 0.3627175829845052, 0.36647819300704754]
Implied volatilities [Merton Jump Diffusion]: [0.6672784188699088, 0.6662112902897455, 0.6651620568297255, 0.6641302856710489, 0.6631155585401313, 0.6621174709777295, 0.6611356316619765, 0.6601696617803263, 0.6592191944459197, 0.6582838741543455, 0.657363356277163, 0.6564573065889483, 0.6555654008249057, 0.6546873242664384]


## Calibration of Merton Jump Diffusion parameters
First we'll initialize a starting point in the array $x0 = [σ, λ, m, v]$
and we set bounds for the 4 parameters. Then we use the method
1. **curve_fit** of scipy.optimize. It uses the trf method when the parameters are bounded and the Levemberg-Marquadt method for unbounded parameters.

In [36]:
x0 = [0.15, 1, 0.1, 1] # initial guess: [σ, λ, m, v]
bounds = ( [0.01, 0, 1e-5, 0], [3, np.inf, 5, 5] )

def f_Mert(x, sigma, lambd, meanJ, stdJ):
    Mert = Merton_pricer(S0=S0, K=x, ttm=T, r=r, sigma=sigma, lambd=lambd, meanJ=meanJ, stdJ=stdJ, exercise=exercise, type_o='call')
    return Mert.closed_formula(x)
#

- Method 1. curve_fit of scipy.optimize. It uses the trf method when the parameters are bounded and the Levemberg-Marquadt method for unbounded parameters.

In [37]:
res1 = scpo.curve_fit(f_Mert, call_strikes, call_prices, p0=x0, bounds=bounds)

sigt = round(res1[0][0],5)
lambdt = round(res1[0][1],2)
mt = round(res1[0][2],2)
vt = round(res1[0][3],5)

print('> METHOD 1: CURVE_FIT (trf)')
print('Calibrated Volatlity [σ] = ', sigt)
print('Calibrated Jump intensity [λ] = ', lambdt)
print('Calibrated Jump Mean = ', mt)
print('Calibrated Jump St. dev.  = ', vt)

> METHOD 1: CURVE_FIT (trf)
Calibrated Volatlity [σ] =  0.01
Calibrated Jump intensity [λ] =  0.43
Calibrated Jump Mean =  1.1
Calibrated Jump St. dev.  =  0.0027


In [38]:
res1_2 = scpo.curve_fit(f_Mert, call_strikes, call_prices, p0=x0)

sigt2 = round(res1_2[0][0],5)
lambdt2 = round(res1_2[0][1],2)
mt2 = round(res1_2[0][2],2)
vt2 = round(res1_2[0][3],5)

print('> METHOD 1.2: CURVE_FIT (Levemberg-Marquadt)')
print('Calibrated Volatlity [σ] = ', sigt2)
print('Calibrated Jump intensity [λ] = ', lambdt2)
print('Calibrated Jump Mean = ', mt2)
print('Calibrated Jump St. dev.  = ', vt2)

C:\Users\ornel\OneDrive\Desktop\TESI\PythonProjects\Margin-Loans-Pricing\python-modules\functions\MERTONpricer.py:49: RuntimeWarning: invalid value encountered in log
  r_k = self.r - self.lambd*(self.meanJ-1) + (k * np.log(self.meanJ)) / self.ttm


> METHOD 1.2: CURVE_FIT (Levemberg-Marquadt)
Calibrated Volatlity [σ] =  -0.14044
Calibrated Jump intensity [λ] =  -0.07
Calibrated Jump Mean =  0.0
Calibrated Jump St. dev.  =  0.2125


c:\users\ornel\appdata\local\programs\python\python38\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


- Method 2. minimize(method=’SLSQP’)

In [39]:
x0 = [0.2, 0.8, 0.1, 1] # initial guess: [σ, λ, m, v]
bounds = [(0.01, 3), (0, np.inf), (1e-5, 5),  (0, 5)]

# Define the objective function
def obj_function(x, K, call_prices):
    sigm, lamb, mean, std = x
    Mert = Merton_pricer(S0=S0, K=K, ttm=T, r=r, sigma=sigm, lambd=lamb, meanJ=mean, stdJ=std, exercise=exercise, type_o='call')
    return np.sum((Mert.closed_formula(K) - call_prices) ** 2)

additional_args = (call_strikes, call_prices)

res2 = scpo.minimize(obj_function, x0, args=additional_args,  method='SLSQP', bounds=bounds, tol=1e-20)

print(res2)
sigt, lambdt, mt, vt = res2.x

# Print the results
print('> METHOD 2: MINIMIZE (SLSQP)')
print('Calibrated Volatility [σ] =', round(sigt, 5))
print('Calibrated Jump Intensity [λ] =', round(lambdt, 2))
print('Calibrated Jump Mean [m] =', round(mt, 2))
print('Calibrated Jump St. dev. [v] =', round(vt, 2))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1831705.969667116
       x: [ 1.452e-01  5.693e-01  1.513e+00  7.117e-01]
     nit: 6
     jac: [ 1.981e+06  4.962e+06  3.445e+06  3.245e+06]
    nfev: 12
    njev: 2
> METHOD 2: MINIMIZE (SLSQP)
Calibrated Volatility [σ] = 0.14522
Calibrated Jump Intensity [λ] = 0.57
Calibrated Jump Mean [m] = 1.51
Calibrated Jump St. dev. [v] = 0.71
